# Import

In [1]:
// %install-swiftpm-flags -c release
// %install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.5")' SwiftCoreMLTools
// %install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

In [2]:
import Foundation
import TensorFlow
// import SwiftCoreMLTools
// import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let embeddingSizes = allCategoriesValues.map{ $0.count }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
let oneHotCategoricalFeatures:[[[Int32]]] = categoricalFeatures.map{ catArray in
    var oneHotArray = [[Int32]]()
    
    for i in 0..<catArray.count {
        var oneHot = Array(repeating: Int32(0), count: allCategoriesValues[i].count)
        if let pos = allCategoriesValues[i].firstIndex(where: { $0 == catArray[i] }){
            oneHot[pos] = 1
        }
        oneHotArray.append(oneHot)
    }
    
    return oneHotArray
}

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Split Train and Test

In [6]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(oneHotCategoricalFeatures[0..<numTrainRecords])).map{ Array($0.joined()) }
let xCategoricalAllTest = matrixTranspose(Array(oneHotCategoricalFeatures[numTrainRecords...])).map{ Array($0.joined()) }
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, embeddingSizes[offset]]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, embeddingSizes[offset]]))
}
let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [7]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[ 2.0137098,  14.197531,   9.523555, 0.53213036,  6.3311296,   64.47929,  4.1678762,  353.68396,
    18.03163,  379.84735,  11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.11449408,  0.7311985,  29.000755,  2.1797554,  132.14561,
    2.217345,  40.494495,   6.852825]]


In [8]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [9]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 2] [405, 9] [405, 1]
Testing shapes  [101, 11] [101, 2] [101, 9] [101, 1]


# Model

In [10]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
    var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
    var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
    var embeddingLayer = Dense<Float>(inputSize: (4 + 45), outputSize: 64, activation: relu)
    var allInputConcatLayer = Dense<Float>(inputSize: (32 + 64), outputSize: 128, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
        let numericalInput = numericalLayer(input.numerical[0])
        let embeddingOutput1 = embedding1(input.categorical[0])
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
        let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        let embeddingInput = embeddingLayer(embeddingConcat)
        let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [11]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [12]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [13]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 538.7566, MAE: 20.769682
Epoch 2: MSE: 326.60144, MAE: 14.487716
Epoch 3: MSE: 174.02321, MAE: 9.4455385
Epoch 4: MSE: 113.11984, MAE: 7.26101
Epoch 5: MSE: 97.41988, MAE: 6.8143406
Epoch 6: MSE: 85.93188, MAE: 6.2576895
Epoch 7: MSE: 75.830185, MAE: 5.9648123
Epoch 8: MSE: 69.65385, MAE: 5.545283
Epoch 9: MSE: 60.797016, MAE: 5.333738
Epoch 10: MSE: 56.28574, MAE: 4.696577
Epoch 11: MSE: 51.698994, MAE: 4.681757
Epoch 12: MSE: 48.156887, MAE: 4.2115726
Epoch 13: MSE: 41.7582, MAE: 4.0844197
Epoch 14: MSE: 41.818134, MAE: 3.9483097
Epoch 15: MSE: 34.868656, MAE: 3.7961638
Epoch 16: MSE: 38.032547, MAE: 3.6246533
Epoch 17: MSE: 33.77977, MAE: 3.5818517
Epoch 18: MSE: 33.526505, MAE: 3.4123552
Epoch 19: MSE: 32.165363, MAE: 3.4164927
Epoch 20: MSE: 30.53839, MAE: 3.3854785
Epoch 21: MSE: 29.151405, MAE: 3.3084984
Epoch 22: MSE: 28.262146, MAE: 3.274151
Epoch 23: MSE: 28.250547, MAE: 3.2525787
Epoch 24: MSE: 28.393291, MAE: 3.1756794
Epoch 25: MSE: 25.969215, MAE: 3.1627727


Epoch 203: MSE: 4.4322333, MAE: 1.4914743
Epoch 204: MSE: 5.102226, MAE: 1.5785955
Epoch 205: MSE: 5.5810084, MAE: 1.5072623
Epoch 206: MSE: 5.068066, MAE: 1.5116196
Epoch 207: MSE: 5.0839076, MAE: 1.4730271
Epoch 208: MSE: 4.3763723, MAE: 1.4861579
Epoch 209: MSE: 4.1561007, MAE: 1.5250263
Epoch 210: MSE: 5.526101, MAE: 1.4812418
Epoch 211: MSE: 5.6118298, MAE: 1.5378153
Epoch 212: MSE: 4.247137, MAE: 1.4208779
Epoch 213: MSE: 5.320103, MAE: 1.5671417
Epoch 214: MSE: 4.6118116, MAE: 1.4267943
Epoch 215: MSE: 4.415875, MAE: 1.5514921
Epoch 216: MSE: 5.1508203, MAE: 1.4871061
Epoch 217: MSE: 4.1682534, MAE: 1.4328892
Epoch 218: MSE: 5.8022723, MAE: 1.4710673
Epoch 219: MSE: 4.4486613, MAE: 1.4278293
Epoch 220: MSE: 4.655449, MAE: 1.4594887
Epoch 221: MSE: 5.071363, MAE: 1.4734396
Epoch 222: MSE: 4.117449, MAE: 1.4228147
Epoch 223: MSE: 4.5146456, MAE: 1.569769
Epoch 224: MSE: 4.3864665, MAE: 1.4835556
Epoch 225: MSE: 4.025299, MAE: 1.4285905
Epoch 226: MSE: 4.1691127, MAE: 1.5233592
Epo

Epoch 402: MSE: 2.8664737, MAE: 1.3277223
Epoch 403: MSE: 2.6558275, MAE: 1.0866882
Epoch 404: MSE: 2.427057, MAE: 1.2064211
Epoch 405: MSE: 2.6761448, MAE: 1.102355
Epoch 406: MSE: 2.384178, MAE: 1.3494707
Epoch 407: MSE: 2.708345, MAE: 1.1365261
Epoch 408: MSE: 2.148976, MAE: 1.1808168
Epoch 409: MSE: 2.9321685, MAE: 1.3322326
Epoch 410: MSE: 2.278824, MAE: 1.273396
Epoch 411: MSE: 2.9815178, MAE: 1.224273
Epoch 412: MSE: 2.2886305, MAE: 1.1418269
Epoch 413: MSE: 2.818647, MAE: 1.2507799
Epoch 414: MSE: 2.4442058, MAE: 1.1488692
Epoch 415: MSE: 2.364881, MAE: 1.3078539
Epoch 416: MSE: 3.3966343, MAE: 1.3626841
Epoch 417: MSE: 1.9701341, MAE: 0.9753904
Epoch 418: MSE: 2.2934823, MAE: 1.2918925
Epoch 419: MSE: 2.8187218, MAE: 1.3370166
Epoch 420: MSE: 2.7583895, MAE: 1.1940745
Epoch 421: MSE: 2.4790819, MAE: 1.0520953
Epoch 422: MSE: 2.2844384, MAE: 1.2423292
Epoch 423: MSE: 2.4127598, MAE: 1.2542771
Epoch 424: MSE: 2.373144, MAE: 1.2433466
Epoch 425: MSE: 2.2326548, MAE: 0.94958335
Ep

# Test

In [14]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.44593185, MAE: 0.046968672


# Export

In [15]:
// print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
// print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

In [16]:
// let coremlModel = Model(version: 4,
//                         shortDescription: "Regression",
//                         author: "Jacopo Mangiavacchi",
//                         license: "MIT",
//                         userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
//     Input(name: "input", shape: [13])
//     Output(name: "output", shape: [1])
//     NeuralNetwork {
//         InnerProduct(name: "dense1",
//                      input: ["input"],
//                      output: ["outDense1"],
//                      weight: model.layer1.weight.transposed().flattened().scalars,
//                      bias: model.layer1.bias.flattened().scalars,
//                      inputChannels: 13,
//                      outputChannels: 64)
//         ReLu(name: "Relu1",
//              input: ["outDense1"],
//              output: ["outRelu1"])
//         InnerProduct(name: "dense2",
//                      input: ["outRelu1"],
//                      output: ["outDense2"],
//                      weight: model.layer2.weight.transposed().flattened().scalars,
//                      bias: model.layer2.bias.flattened().scalars,
//                      inputChannels: 64,
//                      outputChannels: 32)
//         ReLu(name: "Relu2",
//              input: ["outDense2"],
//              output: ["outRelu2"])
//         InnerProduct(name: "dense3",
//                      input: ["outRelu2"],
//                      output: ["output"],
//                      weight: model.layer3.weight.transposed().flattened().scalars,
//                      bias: model.layer3.bias.flattened().scalars,
//                      inputChannels: 32,
//                      outputChannels: 1)
//     }
// }

In [17]:
// let coreMLData = coremlModel.coreMLData
// try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))